In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
%matplotlib inline

# ignore pandas warnings
import warnings
warnings.simplefilter('ignore')

import pickle
import time
start = time.time()

In [2]:
# load data
data = pd.read_csv('training_ultrasound.csv')

# remove agedays > 0 ( we just only focus pre-birth measurements)
data = data[data['AGEDAYS']<0]

# drop rows with missing data in any of the 5 main columns
ultrasound = ['HCIRCM', 'ABCIRCM', 'BPDCM', 'FEMURCM']
aux_measure = ['GAGEDAYS', 'SEXN', 'PARITY', 'GRAVIDA']
target = 'BWT_40'
data.dropna(subset=ultrasound+[target], inplace=True)

# correct faulty data
data.loc[data['STUDYID']==2, 'PARITY'] = data.loc[data['STUDYID']==2, 'PARITY'] + 1

## Model

In [3]:
# select basic vars
df = data[ultrasound + aux_measure + [target]]

In [4]:
df.isnull().sum()

HCIRCM        0
ABCIRCM       0
BPDCM         0
FEMURCM       0
GAGEDAYS      0
SEXN          0
PARITY      101
GRAVIDA     101
BWT_40        0
dtype: int64

In [5]:
# there is missing data for parity and gravida: this happens for first pregnancy --> fill with 1s
df.fillna(1, inplace=True)

# replace sex values to 0 and 1
df['SEXN'] = df['SEXN'].replace([1,2], [0,1])

### Feature engineering 

In [6]:
ultrasound

['HCIRCM', 'ABCIRCM', 'BPDCM', 'FEMURCM']

In [7]:
length_ratios = ['HCIRCM / ABCIRCM', 'HCIRCM / BPDCM', 'HCIRCM / FEMURCM',
                 'ABCIRCM / BPDCM', 'ABCIRCM / FEMURCM', 
                 'BPDCM / FEMURCM']
for ratio in length_ratios:
    df[ratio] = df[ratio.split(' ')[0]] / df[ratio.split(' ')[2]]

In [8]:
lenght_time = list()
for m in ultrasound:
    col_name = '%s / GAGEDAYS' % m
    lenght_time.append(col_name)
    df[col_name] = df[m] / df['GAGEDAYS']

lenght_time

['HCIRCM / GAGEDAYS',
 'ABCIRCM / GAGEDAYS',
 'BPDCM / GAGEDAYS',
 'FEMURCM / GAGEDAYS']

In [9]:
# no of past pregancies
df['past_gest'] = df['PARITY'] - df['GRAVIDA']

other_feat = ['past_gest'] 

In [10]:
# common models for sonographic fetal weight estimation use log of the weight
df['BWT_40'] = np.log(1 + df['BWT_40'])

In [11]:
print('Dataframe size: %s,%s' % (df.shape[0],df.shape[1]))

Dataframe size: 7928,20


In [12]:
# sklearn imports
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error

# xgboost
from xgboost import XGBRegressor

# custom 'library'
from aux_fun import plot_learning_curve, plot_validation_curve

### Split train/test data

In [13]:
pf = PolynomialFeatures(degree=4)
X_poly_ultrasounds = pf.fit_transform(df[ultrasound].values)
X_aux_measure = df[aux_measure].values
X_lenght_ratios = df[length_ratios].values
X_lenght_time = df[lenght_time].values
X_other_feat = df[other_feat].values

X = np.concatenate((X_poly_ultrasounds,X_aux_measure,X_lenght_ratios,X_lenght_time,X_other_feat),axis=1)

Y = df[target].values

In [14]:
poly_feat_names = [e.replace('x0','HCIRCM').replace('x1','ABCIRCM').replace('x2','BPDCM').replace('x3','FEMURCM')
              for e in pf.get_feature_names()]

In [15]:
all_feat_names = pd.Series(poly_feat_names + aux_measure + length_ratios + lenght_time + other_feat)

In [16]:
# train-test split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

### Define model pipeline

In [17]:
xgb = XGBRegressor(max_depth=10,learning_rate=0.1,n_estimators=100,subsample=0.9)

CV using all features:

In [18]:
kf = KFold(n_splits=5)

In [19]:
mae = list()

for train_k, test_k in kf.split(x_train):
    xgb.fit(x_train[train_k],y_train[train_k])
    mae.append(mean_absolute_error(y_train[test_k], xgb.predict(x_train[test_k])))
    
base_cv_mae = np.mean(mae)

print('Score: %0.4f +- %0.4f' % (np.mean(mae),2*np.std(mae)))

Score: 0.0527 +- 0.0027


Iterate through all features, calculating the mean absolute error of the model when leaving that one feature out:

In [20]:
mae_after_deleting = list()
for ix, feat in all_feat_names.iteritems():
    mae_i = cross_val_score(xgb, X=np.delete(x_train,[ix],axis=1), y=y_train, scoring='mean_absolute_error', cv=kf, n_jobs=-1)
    mae_after_deleting.append(mae_i)
    if (ix + 1) % 10 == 0:
        print('step %i done' % (ix + 1))
print('All done!')

step 10 done
step 20 done
step 30 done
step 40 done
step 50 done
step 60 done
step 70 done
step 80 done
All done!


Order features (features with POSITIVE error increments are more important, i.e., excluding them from the model decreases performance):

In [21]:
feature_deltas = [np.abs(mae.mean()) - base_cv_mae for mae in mae_after_deleting]

sorted_feat_importances = \
pd.DataFrame({'names': all_feat_names, 'deltas': feature_deltas})[['names','deltas']].sort_values(by='deltas', ascending=False)

In [22]:
sorted_feat_importances.to_excel('feature_importances.xlsx')
sorted_feat_importances

,names,deltas
71,SEXN,0.000589
73,GRAVIDA,0.000030
76,HCIRCM / FEMURCM,-0.000065
41,HCIRCM^2 ABCIRCM FEMURCM,-0.000145
48,HCIRCM ABCIRCM BPDCM^2,-0.000149
7,HCIRCM BPDCM,-0.000153
80,HCIRCM / GAGEDAYS,-0.000160
57,ABCIRCM^3 FEMURCM,-0.000181
63,ABCIRCM BPDCM FEMURCM^2,-0.000185
4,FEMURCM,-0.000188


In [23]:
model_objects = {'train': (x_train,y_train), 'test': (x_test,y_test)}
with open('data_train_test.p', 'wb') as handle:
    pickle.dump(model_objects, handle)

In [24]:
time.time() - start

528.7203924655914